In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

Импортируем нужные библиотеки

In [2]:
def get_sim(item_id):
    rat = [(other_item_id, np.linalg.norm(H[:, item_id - 1] - H[:, other_item_id - 1]))
           for other_item_id in ratings["movie_id"].unique()]
    items_ratings = sorted(rat, key=lambda x: x[1])
    return items_ratings[:10]


def get_recom(user_id):
    unique_movies = set(movies.to_numpy())
    for k in range(len(ratings_np)):
        if ratings_np[k][0] == user_id:
            unique_movies.remove(ratings_np[k][1])
    unique_movies_arr = []
    for t in unique_movies:
        unique_movies_arr.append(t)
    new_items_ratings = (W @ H)[user_id - 1][np.array(unique_movies_arr) - 1]
    items_ratings = sorted(list(zip(unique_movies, new_items_ratings)), key=lambda x: x[1], reverse=True)
    return items_ratings[:10]

Два дополнительных метода, чтобы считать симилары для конкретного айтема и рекомендации для конкретного юзера

In [3]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      usecols=['user_id', 'movie_id', 'rating'], engine='python')

movie_info = pd.read_csv('movies.dat', delimiter='::', header=None,
                         names=['movie_id', 'name', 'category'], engine='python')

users = ratings["user_id"]
movies = ratings["movie_id"]
ratings_np = ratings.to_numpy()
users_size = users.max()
items_size = movies.max()
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]
implicit_ratings_np = implicit_ratings.to_numpy()
user_item = sp.coo_matrix(
    (np.ones_like(implicit_ratings["user_id"]), (implicit_ratings["user_id"], implicit_ratings["movie_id"])))
user_item_csr_arr = user_item.tocsr().toarray()

Подгружаем наш датасет

In [4]:
K = 100
eps = 1e-3
lam = 1e-5
count_iter = 150

Подбираем гиперпараметры модели

In [5]:
W = np.random.uniform(0, 1 / np.sqrt(K), size=(users_size, K))
H = np.random.uniform(0, 1 / np.sqrt(K), size=(K, items_size))

Инициализируем матрицы юзеров (W) и айтемов (H)

In [6]:
for t in range(count_iter):
    for user in range(users_size):
        pos_items = set(user_item_csr_arr[user + 1, 1:].nonzero()[0])
        for pos_item in pos_items:
            pos_item -= 1
            while True:
                neg_item = np.random.randint(items_size)
                if pos_items.__contains__(neg_item + 1):
                    continue
                else:
                    break
            pos_V = W[user, :] @ H[:, pos_item]
            neg_V = W[user, :] @ H[:, neg_item]
            e = 1 / (1 + np.exp(pos_V - neg_V))
            W[user, :] += eps * (e * (H[:, pos_item] - H[:, neg_item]).T - lam * W[user, :])
            H[:, pos_item] += eps * (e * W[user, :].T - lam * H[:, pos_item])
            H[:, neg_item] += eps * (e * W[user, :].T - lam * H[:, neg_item])

Обучаем нашу модель на implicit данных

In [7]:
get_similars = lambda item_id, model: [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() for x in model]
for i in get_similars(1, get_sim(1)):
    print(i)

0    Toy Story (1995)
3045    Toy Story 2 (1999)
33    Babe (1995)
584    Aladdin (1992)
2286    Bug's Life, A (1998)
2252    Pleasantville (1998)
1132    Wrong Trousers, The (1993)
2618    Tarzan (1999)
360    Lion King, The (1994)
735    Close Shave, A (1995)


Симилары действительно симилары

In [8]:
get_recommendations = lambda user_id, model: [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() for x in model]
for i in get_recommendations(4, get_recom(4)):
    print(i)

585    Terminator 2: Judgment Day (1991)
847    Godfather, The (1972)
1271    Indiana Jones and the Last Crusade (1989)
1182    Aliens (1986)
2502    Matrix, The (1999)
1203    Godfather: Part II, The (1974)
453    Fugitive, The (1993)
1284    Butch Cassidy and the Sundance Kid (1969)
2460    Planet of the Apes (1968)
1250    Back to the Future (1985)


Рекомендации вышли хорошие